## Impact of Lambda
In this part of the notebook, we will estimate $\theta$ tilde under different $\lambda \in \Lambda$. We define a list containing different $\lambda$ and we will create a DataFrame that computes the relative error for every component of $\theta$ several times, and we will look at the mean and standard deviation of the relative error for every component.

In [ ]:
Lambdas = 1e-2*[0.1,0.5,1,5,10,50,100,500,1000]
#np.arange(0.5,5,0.5)#[0.1,0.5,0.8,1,1.3,1.6,2,4,7,9]
nb_sim = 5
method = "source"
n = len(Lambdas)
sim_theta_tildes = np.zeros((T,nb_sim,n))
Y_simu=Create_DicoY(T,Lambdas,a,b)

for k in range(n):
    Y = Y_simu[Lambdas[k]]
    for j in range(nb_sim):
        _, sim_theta_tildes[:,j,k] = MetropolisHastingsFast(T, Lambdas[k], Y,a,b,method=method)
        #print("working...")
    print(f"Oooh yeah! Finished {k+1}. iteration")

In [ ]:
print(pd.DataFrame(sim_theta_tildes[:,:,-1]))

The simulated $\theta$ sometimes takes tiny values close to 0, although the theoretical value is a bigger real number. This means that the relative error can exaggerate the magnitude of the error, since we divide by a small number. Therefore we opt for the following error:
$d(x,y) = \frac{|x-y|}{max(|x|,|y|)}$

In [ ]:
def dist(x,y):
    return np.abs(x-y)/np.maximum(np.abs(x),np.abs(y))

In [ ]:
errors = np.zeros(np.shape(sim_theta_tildes))
for k in range(n):
    Y = Y_simu[Lambdas[k]]
    _, theoretical_means = ComputeMeans(T, Lambdas[k], Y, a, b)
    errors[:,:,k] = dist(sim_theta_tildes[:,:,k],theoretical_means[:,np.newaxis] @ np.ones((1,nb_sim)))
    #errors[:,:,k] = abs(sim_theta_tildes[:,:,k] - theoretical_means[:,np.newaxis] @ np.ones((1,10))) / np.max(np.abs(sim_theta_tildes[:,:,k])

In [ ]:
print(np.shape(errors))
data = np.zeros((T,2*n))
for i in range(n//2+1):
    data[:,2*i] = np.mean(errors[:,:,i],axis=1)
    data[:,2*i+1]=np.std(errors[:,:,i],axis=1)
assert np.shape(data)==(T,2*n)
df = pd.DataFrame(data)

df.columns = pd.MultiIndex.from_product([Lambdas,["mean","std"]],names=["lambdas","measure"])

In [ ]:
df.head()

In [ ]:
df_mean = df.mean()

In [ ]:
niter = 10**5
fig,ax = plt.subplots(1,1)
df_mean.xs("mean", level=1, axis=0).plot(label="emp. mean")
df_mean.xs("std",  level=1, axis=0).plot(label="standard dev.")
fig.suptitle(f"Error as function of $\lambda$, @{niter:} iterations, using method: {method}")
ax.set_ylabel("distance")
ax.legend()
plt.grid()

In [ ]:
T = 30
Lambdas = [1] + [5*i for i in range(1,5)] + [50,100]
n_simu = len(Lambdas)
theoretical_means = np.zeros((n_simu, T))
errors_source = np.zeros(n_simu)
errors_image = np.zeros(n_simu)
errors_subdiff_source = np.zeros(n_simu)
errors_subdiff_image = np.zeros(n_simu)
errors_prox_image = np.zeros(n_simu)

for i in range(n_simu):
    #print(ComputeMeans(T, 1, Y_simu[Lambdas[i]])[0])
    theoretical_means[i,:] = ComputeMeans(T, 1, Y_simu[Lambdas[i]], a, b)[0]
    #print("done1")
    errors_source[i] = sum(abs(MetropolisHastingsFast(T, 1, Y_simu[Lambdas[i]], a, b, method="source")[0]-theoretical_means[i,:]))
    #print(errors_source[i])
    errors_image[i] = sum(abs(MetropolisHastingsFast(T, 1, Y_simu[Lambdas[i]], a, b, method="image")[0]-theoretical_means[i,:]))
    #print(errors_image[i])
    errors_subdiff_source[i] = sum(abs(MetropolisHastingsFast(T, 1, Y_simu[Lambdas[i]], a, b, method="subdiff_source")[0]-theoretical_means[i,:]))
    #print(errors_subdiff_source[i])
    errors_subdiff_image[i] = sum(abs(MetropolisHastingsFast(T, 1, Y_simu[Lambdas[i]], a, b, method="subdiff_image")[0]-theoretical_means[i,:]))
    #print(errors_subdiff_image[i])
    errors_prox_image[i] = sum(abs(MH_Prox_Image(T, 1, Y_simu[Lambdas[i]], a, b)[0][-1]-theoretical_means[i,:]))
    print("---Done---")

plt.figure()
plt.plot(errors_source, color = "blue")
plt.plot(errors_image, color = "green")
plt.plot(errors_subdiff_source, color = "red")
plt.plot(errors_subdiff_image, color = "black")
plt.plot(errors_prox_image, color = "yellow")
plt.show()